In [1]:
import pandas as pd
import seaborn as sns

In [3]:
# 주식 데이터를 가져와서 데이터 프레임 만들기

df1 = pd.read_excel('./stock_price.xlsx')
df2 = pd.read_excel('./stock_valuation.xlsx')

In [5]:
df2.count()

id      10
name    10
eps     10
bps     10
per     10
pbr     10
dtype: int64

In [13]:
#데이터 프레임 합치기 (교집합)
merge_inner = pd.merge(df1, df2)
merge_inner

#데이터 프레임 합치기 (합집합)
merge_outer = pd.merge(df1, df2, how = 'outer', on = 'id')
merge_outer

#데이터 프레임 합치기 (왼쪽 프레임 기준,키 값 분리, how = 'left', left_on=칼럼명)
merge_left = pd.merge(df1, df2, how = 'left', left_on = 'id', right_on = 'id')
merge_left

# 불린 인덱싱과 결합하여 원하는 데이터 찾기
price = df1[df1['price']<50000]
value = pd.merge(price, df2)
value

,id,stock_name,value,price,name,eps,bps,per,pbr
0,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [14]:
df1.set_index('id', inplace=True)
df2.set_index('id', inplace=True)

In [15]:
# 데이터프레임 결합(join) - 교집합 인덱스로 병합
df3 = df1.join(df2, how='inner') # how = inner, outer, right, left
df3

,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [19]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]
len(df)

# class 열을 기준으로 분할
grouped = df.groupby(['class'])
grouped.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
6,54.0,male,First,51.8625,0
7,2.0,male,Third,21.0750,0
9,14.0,female,Second,30.0708,1
11,58.0,female,First,26.5500,1


In [24]:
df = titanic.loc[:,['age','sex','class','fare','survived']]

# class 열을 기준으로 분할
grouped = df.groupby(['class'])

# 각 그룹에 대한 모든 열의 표준편차를 집게하여 데이터프레임으로 반환
std_all = grouped.std()
std_all

# 각 그룹에 대한 fare 열의 표준편차를 집게하여 시리즈로 변환
grouped.fare.std()

class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64

In [33]:
grouped_two = df.groupby(['class', 'sex'])
# grouped_two 그룹 객체에 연산 메소드 적용

average_two = grouped_two.mean()
average_two

grouped_two.get_group(('Third', 'female'))

#여러 함수를 각 열에 동일하게 적용하여 집계
agg_all= grouped.agg(['min','max'])
print(agg_all)

# 각 열마다 다른 함수를 적용하여 집계
agg_aal= grouped.agg({'fare':['min','max'], 'age':'mean'})
agg_aal

a = grouped.filter(lambda x: len(x) > 200)
b = grouped.filter(lambda x: x.age.mean()<30)

         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1


In [42]:
grouped = df.groupby(['class', 'sex'])

gdf = grouped.mean()
gdf

#class 값이 First인 행을 선택
gdf.loc['First']

#class 값이 First이고 sex값이 male
gdf.loc[('First','male')]

#sex 값이 male인 행 선택 : 그룹함수결과.xs(그룹칼럼의 값, level = 그룹칼럼명)
gdf.xs('male', level='sex')

,age,fare,survived
class,,,
First,41.281386,67.226127,0.368852
Second,30.740707,19.741782,0.157407
Third,26.507589,12.661633,0.135447


In [49]:
# 행, 열, 값, 집계에 사용 할 열을 하나씩 지정 - 평균집계
pdf1 = pd.pivot_table(df, # 피벗 할 데이터 프레임
                    index = 'class', # 행의 위치로 들어갈 열
                    columns = 'sex', # 열의 위치 들어갈 열
                    values = 'age', # 데이터로 사용 할 열
                    aggfunc = 'mean') # 데이터 집계 함수

pdf2 = pd.pivot_table(df,
                     index = 'class',
                     columns = 'sex',
                     values = 'age',
                     aggfunc = ['mean', 'sum'])

pdf2.xs('First')

      sex   
mean  female      34.611765
      male        41.281386
sum   female    2942.000000
      male      4169.420000
Name: First, dtype: float64